# Brownian Motion

Create simulation

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import torch
from pygsig.graph import StaticGraphTemporalSignal
import torch_geometric.transforms as T
from pygsig.signature import SignatureFeatures, StatFeatures

class Simulation():
    def __init__( self,num_nodes,num_blocks, p_across_blocks,p_within_blocks, mu_gain,beta_gain,sigma_gain,omega_noise,time_horizon,dt = 1e-3):
        self.num_nodes = num_nodes
        self.num_blocks = num_blocks
        self.p_across_blocks = p_across_blocks
        self.p_within_blocks = p_within_blocks
        self.mu_gain = mu_gain
        self.beta_gain = beta_gain
        self.sigma_gain = sigma_gain
        self.omega_noise = omega_noise
        self.time_horizon = time_horizon
        self.dt = dt
        self.num_time_steps = int(time_horizon / dt)

    def run(self,graph_seed,omega_seed,param_seed):

        # synchronization
        def kuramoto(graph, theta, omega, dt):
            dtheta = omega * dt  # Initialize with intrinsic frequencies
            for u, v, data in graph.edges(data=True):
                coupling = data['weight']
                dtheta[u] += dt * coupling * np.sin(theta[v] - theta[u])
                dtheta[v] += dt * coupling * np.sin(theta[u] - theta[v])
            return theta + dtheta

        # drift of the SDE
        def periodic_drift(beta, theta, omega, mu_0, t):
            return mu_0 + beta*np.sin(omega*t + theta)

        # Create a graph
        block_sizes = [self.num_nodes // self.num_blocks] * self.num_blocks
        block_probs = np.zeros((self.num_blocks, self.num_blocks))

        for i in range(self.num_blocks):
            for j in range(self.num_blocks):
                if i == j:
                    block_probs[i, j] = self.p_within_blocks
                else:
                    block_probs[i, j] = self.p_across_blocks
        
        graph = nx.stochastic_block_model(block_sizes, block_probs, seed=graph_seed)

        for edge in graph.edges:
            if graph.nodes[edge[0]]['block'] == graph.nodes[edge[1]]['block']:
                graph[edge[0]][edge[1]]['weight'] = 1/(np.sqrt(graph.degree[edge[0]]*graph.degree[edge[1]]))
            else:
                graph[edge[0]][edge[1]]['weight'] = 1/(np.sqrt(graph.degree[edge[0]]*graph.degree[edge[1]]))
        
        # Assign omega to each node
        np.random.seed(omega_seed)
        omega_range = np.linspace(0,1, self.num_blocks+1)[1:]
        for node in graph.nodes:
            graph.nodes[node]['omega'] = omega_range[graph.nodes[node]['block']] + self.omega_noise * np.random.randn()
        
        # Othe oscilator perameters
        np.random.seed(param_seed)
        omega = np.array([graph.nodes[node]['omega'] for node in graph.nodes])
        beta =  self.beta_gain * np.random.rand(self.num_nodes) # amplitude (uniform nodes)
        theta = 2 * np.pi * np.random.rand(self.num_nodes)  # initial phase (random across nodes)
        mu_0 = self.mu_gain * np.random.rand(self.num_nodes)

        # initial values
        X = np.random.rand(self.num_nodes) # signal

        # Simulate
        theta_traj = np.zeros((self.num_nodes,self.num_time_steps))
        mu_traj = np.zeros((self.num_nodes,self.num_time_steps))
        X_traj = np.zeros((self.num_nodes,self.num_time_steps))

        # Time sequence
        tt = np.arange(0, self.time_horizon, self.dt)
        for step,t in enumerate(tt):
            theta_traj[:, step] = theta
            if step == 0:
                mu_traj[:,step] = mu_0
            else:
                mu_traj[:,step] = mu
            X_traj[:,step] = X
            theta = kuramoto(graph, theta, omega,self.dt)
            mu = periodic_drift(beta, theta,omega, mu_0, t)
            X = X + self.dt * mu + np.sqrt(self.dt) * self.sigma_gain*np.random.randn(self.num_nodes)

        return X_traj, theta_traj, mu_traj, omega, mu_0, beta,  graph


In [2]:
def get_sequence(X_traj,graph):
    snapshot_count = X_traj.shape[1]
    df_edge = nx.to_pandas_edgelist(graph.to_directed())
    edge_index = torch.tensor(df_edge[['source','target']].values.T,dtype=torch.long)
    edge_weight = torch.tensor(df_edge['weight'].values,dtype=torch.float)
    snapshot_count = X_traj.shape[1]
    features = [ torch.tensor(X_traj[:,t],dtype=torch.float).unsqueeze(-1) for t in range(snapshot_count)]
    targets = [ torch.tensor(np.array([graph.nodes[node]['omega'] for node in graph.nodes]),dtype=torch.float).unsqueeze(-1) for _ in range(snapshot_count)]
    # Sequential Data
    return StaticGraphTemporalSignal(edge_index=edge_index,edge_weight=edge_weight,features=features,targets=targets)

Test simulation

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as mcolors

simulation =   Simulation(num_nodes= 20,
                num_blocks=2,
                p_across_blocks=0.1,
                p_within_blocks=0.5,
                mu_gain= 1.0,
                beta_gain=5.0,
                sigma_gain=1.0,
                omega_noise=0.1,
                time_horizon=10,
                dt=1e-3)

X_traj, theta_traj, mu_traj, omega, mu_0, beta, graph = simulation.run(graph_seed=32,omega_seed=29,param_seed=29)

In [ ]:
import networkx as nx

import matplotlib.pyplot as plt

# Get the block information for each node
blocks = nx.get_node_attributes(graph, 'block')

# Create a color map for the blocks
unique_blocks = list(set(blocks.values()))
colors = ['#012169','#4B9CD3']
block_color_map = {block: colors[i] for i, block in enumerate(unique_blocks)}

# Assign colors to nodes based on their block
node_colors = [block_color_map[blocks[node]] for node in graph.nodes]

# Draw the graph
plt.figure(figsize=(4, 4),dpi=200)
nx.draw(graph, node_color=node_colors,pos=nx.circular_layout(graph), node_size=100, font_size=10, font_color='white')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as mcolors

norm = mcolors.Normalize(vmin=0, vmax=1)

fig, ax = plt.subplots(figsize=(10, 4),dpi=200)
cmap = cm.get_cmap('Spectral')


for node in range(simulation.num_nodes):
    sns.lineplot(x=np.linspace(0, simulation.time_horizon, simulation.num_time_steps), 
                 y=X_traj[node, :], 
                 color=block_color_map[blocks[node]], 
                 ax=ax)

# Create ScalarMappable for colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

plt.xlabel('$t$')
plt.ylabel('$X$')
plt.show()

Make dataset consisting of multiple SDE trajectories

In [ ]:
from tqdm import tqdm

simulation =   Simulation(num_nodes= 300,
            num_blocks=3,
            p_across_blocks=0.01,
            p_within_blocks=0.1,
            mu_gain= 1,
            beta_gain=4,
            sigma_gain=1.0,
            omega_noise=0.0,
            time_horizon=10,
            dt=1e-3)



seq_dataset = []
num_runs = 20

with tqdm(total=num_runs) as pbar:
    for run in range(num_runs):
        paths, theta, mu, omega, mu_0, beta, graph = simulation.run(graph_seed=run,omega_seed=run,param_seed=run)
        seq = get_sequence(paths,graph)
        seq_dataset.append(seq)
        pbar.update(1)

save/load dataset

In [2]:
import torch
filename = 'datasets/brownian/brownian-1d-3b.pt'
# torch.save(seq_dataset,filename)
seq_dataset = torch.load(filename)

Train, baby, train!

In [ ]:
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from signatory import signature_channels


import pygsig.graph 
import pygsig.models
import pygsig.signature
from pygsig.models import GCNRegression, MLPRegression
from pygsig.graph import  split_nodes

import importlib
importlib.reload(pygsig.models)
importlib.reload(pygsig.signature)
importlib.reload(pygsig.graph)

num_nodes = seq_dataset[0].num_nodes
dim = seq_dataset[0].num_node_features
out_channels = 1
num_splits = 5 # split the nodes
num_runs = len(seq_dataset)
num_trials = 4
num_epochs = 200

# hyperparameters
learning_rate = 1e-3
lasso = 0
num_hidden = 64

print_during_training = False

all_models = []
all_model_parameters = []
all_model_depths = []
all_model_layers = []
all_models_mse = []
all_models_mae = []
all_models_rmse = []

# create models 
max_depth = 5
max_layers = 5
for depth in range(1,max_depth+1):
    print(f'Signature depth: {depth}')
    dataset = []
    for seq in tqdm(seq_dataset):
        signature_transform = SignatureFeatures(sig_depth=depth, normalize=True, log_signature=False,lead_lag=True)
        dataset.append(signature_transform(seq))
    for num_layers in range(max_layers):
        in_channels = signature_channels(2*dim,depth)
        model = GCNRegression(num_channels=[in_channels]+num_layers*[num_hidden]+[out_channels])
        num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f'Name: {model._get_name()}')
        print(f"Number of parameters: {num_params}")
        print(f"Number of layers: {num_layers+1}")
        print(f"Signature depth: {depth}")
        print(f"Splits: {num_splits}, Trials: {num_trials}, Runs: {num_runs}, Epochs: {num_epochs}")
        all_model_parameters.append(num_params)
        all_model_layers.append(num_layers+1)
        all_model_depths.append(depth)
        
        criterion = nn.MSELoss() # loss function
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=lasso) 
        train_losses = np.zeros([num_splits, num_runs, num_trials, num_epochs]) 
        eval_losses = np.zeros([num_splits, num_runs, num_trials, num_epochs])
        mse = np.zeros([num_splits, num_runs, num_trials])
        mae = np.zeros([num_splits, num_runs, num_trials])
        rmse = np.zeros([num_splits, num_runs, num_trials])

        with tqdm(total=num_splits*num_runs*num_trials, disable=print_during_training) as pbar:
            splits = split_nodes(num_nodes, num_splits,test_ratio=1.0,seed=29)
            for split in range(num_splits):
                train_indices, eval_indices, test_indices = splits[split]
                train_mask = torch.zeros(num_nodes, dtype=torch.bool)
                eval_mask = torch.zeros(num_nodes, dtype=torch.bool)
                test_mask = torch.zeros(num_nodes, dtype=torch.bool)
                train_mask[train_indices] = True
                eval_mask[eval_indices] = True
                test_mask[test_indices] = True
                for run, data in enumerate(dataset):
                    for trial in range(num_trials):
                        model.reset_parameters()
                        for epoch in range(num_epochs):
                            # train
                            model.train()
                            optimizer.zero_grad()
                            out = model(data.x, data.edge_index)
                            train_loss = criterion(out[train_mask], data.y[train_mask])
                            train_loss.backward()
                            optimizer.step()
                            # evaluate
                            model.eval()
                            with torch.no_grad():
                                eval_loss = criterion(out[eval_mask], data.y[eval_mask])
                                train_losses[split, run, trial, epoch] = train_loss.item()
                                eval_losses[split, run, trial, epoch] = eval_loss.item()
                            if epoch % 10 == 0 and print_during_training:
                                print(f'Split {split}, Run {run}, Trial {trial}, Epoch {epoch}, Train MSE Loss: {train_loss.item():.4f}, Evaulation MSE Loss: {eval_loss.item():.4f}')
                        pbar.update(1)
                        # compute the errors on the testing loss after the last epoch
                        with torch.no_grad():
                            out = model(data.x, data.edge_index)
                            mse[split, run, trial] = mean_squared_error(data.y[test_mask], out[test_mask])
                            mae[split, run, trial] = mean_absolute_error(data.y[test_mask], out[test_mask])
                            rmse[split, run, trial] = np.sqrt(mean_squared_error(data.y[test_mask], out[test_mask]))

        print(f'MSE: {np.mean(mse):.4f} ± {np.std(mse):.4f}, MAE: {np.mean(mae):.4f} ± {np.std(mae):.4f}, RMSE: {np.mean(rmse):.4f} ± {np.std(rmse):.4f}')  
        all_models_mse.append(mse)
        all_models_mae.append(mae)
        all_models_rmse.append(rmse)

        # Plotting
        avg_train_losses = np.mean(train_losses, axis=(0,1,2))
        avg_eval_losses = np.mean(eval_losses, axis=(0,1,2))
        std_train_losses = np.std(train_losses, axis=(0,1,2))
        std_eval_losses = np.std(eval_losses, axis=(0,1,2))
        
        plt.figure()
        plt.plot(avg_train_losses,  label='Train Loss', color='maroon')
        # plt.plot(avg_eval_losses,  label='Evaluation Loss', color='navy')
        plt.fill_between(range(num_epochs), avg_train_losses - std_train_losses, avg_train_losses + std_train_losses, alpha=0.1, color='maroon')
        # plt.fill_between(range(num_epochs), avg_eval_losses - std_eval_losses, avg_eval_losses + std_eval_losses, alpha=0.1,color='navy')
        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.legend()
        plt.show()

Dataframe with all results

In [ ]:
import pandas as pd
import numpy as np

# Collecting data in a list of dictionaries
data = []
# Populate the data list
for idx,model in enumerate(all_models):
    for split in range(num_splits):
        for run in range(num_runs):
            for trial in range(num_trials):
                # Extract MAE and RMSE for the current model, split, and run
                mae_value = all_models_mae[idx][split, run, trial]
                mse_value = all_models_mse[idx][split, run, trial]
                rmse_value = all_models_rmse[idx][split, run, trial]
                num_layers = all_model_layers[idx]
                depth = all_model_depths[idx]
                num_params = all_model_parameters[idx]
                
                # Append the data as a dictionary
                data.append({
                    'SigDepth': depth,
                    'NumLayers': num_layers,
                    'NumParams': num_params,
                    'Split': split,
                    'Run': run,
                    'Trial': trial,
                    'MAE': mae_value,
                    'MSE': mse_value,
                    'RMSE': rmse_value
                })

# Create the DataFrame from the list of dictionaries
df = pd.DataFrame(data)

## Graph Recurrent Convolutional Neutral Networks

In [ ]:
import torch
import torch.nn as nn
import signatory
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch_geometric.transforms as T


import pygsig.graph 
import pygsig.models
import pygsig.signature
from pygsig.models import GConvGRURegression,GConvLSTMRegression
from pygsig.graph import StaticGraphTemporalSignal, split_nodes
from pygsig.signature import SignatureFeatures

import importlib
importlib.reload(pygsig.models)
importlib.reload(pygsig.signature)
importlib.reload(pygsig.graph)

num_splits = 5
num_runs = len(seq_dataset)
num_trials = 1
num_epochs = 10
num_nodes = seq_dataset[0].num_nodes
dim = seq_dataset[0].num_node_features
sample_rate = 100 # only take every 'sample_rate' snapshots
num_snapshots = seq_dataset[0].snapshot_count//sample_rate

learning_rate = 1e-3
lasso = 0
num_hidden = 64

print_during_training = True

# initialize models
models = []
models += [GConvGRURegression(num_channels=[dim, num_hidden, 1],K=3)]
models += [GConvLSTMRegression(num_channels=[dim, num_hidden, 1],K=3)]

mse_models = []
mae_models = []
rmse_models = []

for model in models:
    print(f'Model: {model._get_name()}')
    print(f"Number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=lasso)
    train_losses = np.zeros([num_splits, num_runs, num_trials, num_epochs,num_snapshots])
    eval_losses = np.zeros([num_splits, num_runs, num_trials, num_epochs,num_snapshots])
    mse = np.zeros([num_splits, num_runs, num_trials])
    mae = np.zeros([num_splits, num_runs, num_trials])
    rmse = np.zeros([num_splits, num_runs, num_trials])

    with tqdm(total=num_splits*num_runs*num_trials, disable=print_during_training) as pbar:
        splits = split_nodes(num_nodes, num_splits,test_ratio=1.0,seed=29)
        for split in range(num_splits):
            train_indices, eval_indices, test_indices = splits[split]
            train_mask = torch.zeros(num_nodes, dtype=torch.bool)
            eval_mask = torch.zeros(num_nodes, dtype=torch.bool)
            test_mask = torch.zeros(num_nodes, dtype=torch.bool)
            train_mask[train_indices] = True
            eval_mask[eval_indices] = True
            test_mask[test_indices] = True
            for run, full_seq in enumerate(seq_dataset):
                seq = StaticGraphTemporalSignal(edge_index=full_seq.edge_index,edge_weight=full_seq.edge_weight,features=full_seq.features[::sample_rate],targets=full_seq.targets[::sample_rate])
                for trial in range(num_trials):
                    model.reset_parameters()
                    for epoch in range(num_epochs):
                        for t,snapshot in enumerate(seq):
                            # train
                            model.train()
                            optimizer.zero_grad()
                            out = model(snapshot.x, snapshot.edge_index)
                            train_loss = criterion(out[train_mask], snapshot.y[train_mask])
                            train_loss.backward()
                            optimizer.step()
                            # evaluate
                            model.eval()
                            with torch.no_grad():
                                eval_loss = criterion(out[eval_mask], snapshot.y[eval_mask])
                                train_losses[split, run, trial, epoch,t] = train_loss.item()
                                eval_losses[split, run, trial, epoch,t] = eval_loss.item()
                        if print_during_training:
                            print(f'Split {split}, Run {run}, Trial {trial}, Epoch {epoch}, Train MSE Loss: {train_loss.item():.4f}')
                    pbar.update(1)

                    # compute the errors on the testing loss after the last epoch
                    with torch.no_grad():
                        out = model(snapshot.x, snapshot.edge_index)
                        mse[split, run, trial] = mean_squared_error(snapshot.y[test_mask], out[test_mask])
                        mae[split, run, trial] = mean_absolute_error(snapshot.y[test_mask], out[test_mask])
                        rmse[split, run, trial] = np.sqrt(mean_squared_error(snapshot.y[test_mask], out[test_mask]))

    print(f'MSE: {np.mean(mse):.4f} ± {np.std(mse):.4f}, MAE: {np.mean(mae):.4f} ± {np.std(mae):.4f}, RMSE: {np.mean(rmse):.4f} ± {np.std(rmse):.4f}')  
    mse_models.append(mse)
    mae_models.append(mae)
    rmse_models.append(rmse)


    # Plotting
    avg_train_losses = np.mean(train_losses, axis=(0,1,2,-1))
    avg_eval_losses = np.mean(eval_losses, axis=(0,1,2,-1))
    std_train_losses = np.std(train_losses, axis=(0,1,2,-1))
    std_eval_losses = np.std(eval_losses, axis=(0,1,2,-1))
    
    plt.figure()
    plt.plot(avg_train_losses,  label='Train Loss', color='maroon')
    # plt.plot(avg_eval_losses,  label='Evaluation Loss', color='navy')
    plt.fill_between(range(num_epochs), avg_train_losses - std_train_losses, avg_train_losses + std_train_losses, alpha=0.1, color='maroon')
    # plt.fill_between(range(num_epochs), avg_eval_losses - std_eval_losses, avg_eval_losses + std_eval_losses, alpha=0.1,color='navy')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.legend()
    plt.show()

## Autoencoder